# GHZ state preparation

In [ ]:
import qiskit
# from equal1.benchmarking import qiskit_wrapper
# from equal1.benchmarking import qbraid_wrapper
# import qiskit.circuit
import matplotlib.pyplot as plt
import numpy as np
from equal1.benchmarking.algorithms.ghzstate import GHZStateIndividual 
import pandas as pd
import seaborn as sns

In [ ]:
import qbraid
provider = qbraid.QbraidProvider()
device = "equal1_simulator"
device = provider.get_device(device)
device.profile.model_dump()



In [ ]:
hardware = {'hpc-conservative':"Current Device extrapolated", 'hpc-gamma1': "32 Qubit QPU", 'hpc-gamma2': "167 Qubit QPU"}


In [ ]:
# all_experiments = [GHZStateIndividual(n_qubits=q, device_name="bell2-17-gen-preview") for q in range(2, 9, 2)]
all_experiments = [GHZStateIndividual(n_qubits=q, device_name=device_name) for q in range(2, 17, 2) for device_name in hardware.keys()]
# all_experiments = [GHZStateIndividual(n_qubits=q, device_name=device_name) for q in range(2, 9, 2) for device_name in hardware.keys()]
results  = []
for individual in all_experiments:
    individual.run(simulate=True,  runtime_options={"backend":"TensorNet_MPS", "simulation_platform":"GPU"})
    individual.collect_results()
    fid = individual.analyse_results()
    row = {"n_qubits": individual.n_qubits, "device": individual.device_name, "fidelity": fid}
    print(f"Running for {individual.n_qubits} qubits, device={individual.device_name} results: {fid}")
    results.append(row)


df = pd.DataFrame.from_dict(results)
df.to_csv("ghz_state.csv")


In [ ]:
fig, ax = plt.subplots(figsize=(6, 6))
df["label"] = df["device"].map(hardware)

sns.lineplot(data=df, x="n_qubits", y="fidelity", hue="label", marker="o", ax=ax)
ax.axhline(0.5, ls='--', color='red')
ax.set_ylabel("GHZ Fidelity")
ax.legend(title="Simulated Devices")

ax.set_xlabel("Number of Qubits")
ax.set_title("GHZ State Fidelity vs Number of Qubits")